# Looping Through different variables to find the best combination

## Results
### Starting Data - 1 month
- Best Results
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's =
- Best Combination for XGBoost
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's =
- Best Combination for Logistic Regression
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's = 
- Best Combination for LSTM
    1. Data Order =
    2. Underlying Data Type =
    3. Number of Feilds =
    4. Derivatives =
    5. TA's =
## Process
### Get the Data
1. Start with 1 month of data for speed
2. Continue with 1 year of data
3. Based on the random data vs ordered data, take 1 year of random data
    (Look at the the whole database and pull out a number of random entries equivalent to 1 years worth)
4. Take the most efficient process and apply it to the whole dataset in subgroups, randomly assigning groups. Steps 3 and 4 are to prevent overfitting over a single Ticker

### Processing 
1. I have 7 variables to test
    1. Number of Fields - Number of fields in the packet (lookback period)
    2. Derivative - Using the First and Second Derivative, and using it on:
        - Raw Data
        - Percent Change
        - EMA8, SMA8, EMA20, SMA20
        - Mix of these
    3. EMAs
    4. SMAs
    5. Other
    6. Underlying Data
    7. Data Order
2. Loop Through all of the variables and test the XGBoost on all of them
3. Loop Through all of them and test Logistic Regression
4. Loop Through all of them and test LSTM Neural Network


# Getting Data

# Processing for XGBoost

# Processing for Random Forest

# Processing for LSTM 